# Assignment 2

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Sexism Detection, Multi-class Classification, LLMs, Prompting


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Relevant Material

- Tutorial 3
- Huggingface documentation
- Huggingface hub

# Introduction

You are tasked to address the [EDOS Task B](https://github.com/rewire-online/edos) on sexism detection.

## Problem definition

Given an input text sentence, the task is to label the sentence as non-sexist or one of these four sexist categories: (1) threats, (2)  derogation, (3) animosity, (4) prejudiced discussion.

### Examples:

**Text**: *``Schedule a date with her, then don't show up. Then text her "GOTCHA B___H".''*

**Label**: Threats

**Text**: *``That’s completely ridiculous a woman flashing her boobs is not sexual assault in the slightest.''*

**Label**: Not sexist



## Approach

We will tackle the five-class classification task with LLMs.

In particular, we'll consider zero-/few-shot prompting approaches to assess the capability of some popular open-source LLMs on this task.

## Preliminaries

We are going to download LLMs from [Huggingface](https://huggingface.co/).

Many of these open-source LLMs require you to accept their "Community License Agreement" to download them.

In summary:

- If not already, create an account of Huggingface (~2 mins)
- Check a LLM model card page (e.g., [Mistral v3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)) and accept its "Community License Agreement".
- Go to your account -> Settings -> Access Tokens -> Create new token -> "Repositories permissions" -> add the LLM model card you want to use.
- Save the token (we'll need it later)

### Huggingface Login

Once we have created an account and an access token, we need to login to Huggingface via code.

- Type your token and press Enter
- You can say No to Github linking

In [ ]:
!hf auth login

After login, you can download all models associated with your access token in addition to those that are not protected by an access token.

### Data Loading

Since we are only interested in prompting, we do not require a train dataset.

We have preparared a small test set version of EDOS in our dedicated [Github repository](https://github.com/lt-nlp-lab-unibo/nlp-course-material).

Check the ``Assignment 2/data`` folder.
It contains:

- ``a2_test.csv`` → a small test set of 300 samples.
- ``demonstrations.csv`` -> a batch of 1000 samples for few-shot prompting.

Both datasets contain a balanced number of sexist and not sexist samples.


### Imports

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install evaluate
!pip install bitsandbytes
!pip install sentence_transformers

In [ ]:
import evaluate
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from google.colab import drive
import json
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
from datasets import Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

### Instructions

We require you to:

* **Download** the ``A2/data`` folder.
* **Encode** ``a2_test.csv`` into a ``pandas.DataFrame`` object.

In [ ]:
!git clone https://github.com/nlp-unibo/nlp-course-material.git
!ls nlp-course-material/2025-2026/Assignment\ 2/data

data_path = "nlp-course-material/2025-2026/Assignment 2/data"

In [ ]:
df = pd.read_csv(f"{data_path}/a2_test.csv")
df.head()

The data is perfectly balanced as we can see from the followign code. In fact, each class is equally represented.

In [ ]:
print(df['label_category'].value_counts())

# [Task 1 - 0.5 points] Model setup

Once the test data has been loaded, we have to setup the model pipeline for inference.

In particular, we have to:
- Load the model weights from Huggingface
- Quantize the model to fit into a single-GPU limited hardware

### Which LLMs?

The pool of LLMs is ever increasing and it's impossible to keep track of all new entries.

We focus on popular open-source models.

- [Mistral v2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)
- [Mistral v3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)
- [Llama v3.1](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)
- [Phi3-mini](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)
- [TinyLlama](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0)
- [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B)
- [Qwen3](https://huggingface.co/Qwen/Qwen3-1.7B)

Other open-source models are more than welcome!

### Instructions

In order to get Task 1 points, we require you to:

* Pick 2 model cards from the provided list.
* For each model:
  - Setup a quantization configuration for the model.
  - Load the model via HuggingFace APIs.


### Note

There's a popular library integrated with Huggingface's ``transformers`` to perform quantization.


### Our implementation

We chose to use the Mistral v3 model and the Llama v3.1 using the quantization configuration seen during the tutorial on prompting.

In the following code snippets we load, quantize and print the structure of the two models. We do also load the tokenizers corresponding to the used models.

In [ ]:
from transformers import (AutoTokenizer,
                          DataCollatorWithPadding,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
mistral_card = "mistralai/Mistral-7B-Instruct-v0.3"

mistral_tokenizer = AutoTokenizer.from_pretrained(mistral_card)
mistral_tokenizer.padding_token = mistral_tokenizer.eos_token

mistral_model = AutoModelForCausalLM.from_pretrained(
    mistral_card,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto'
)

# Configure generation for Mistral
mistral_gen = mistral_model.generation_config
mistral_gen.max_new_tokens = 40
mistral_gen.eos_token_id = mistral_tokenizer.eos_token_id
mistral_gen.pad_token_id = mistral_tokenizer.eos_token_id
mistral_gen.temperature = None
mistral_gen.num_return_sequences = 1

print(mistral_model)

In [ ]:
# qwen_card = "Qwen/Qwen3-1.7B"

# qwen_tokenizer = AutoTokenizer.from_pretrained(qwen_card)

# qwen_model = AutoModelForCausalLM.from_pretrained(
#     qwen_card,
#     return_dict=True,
#     quantization_config=bnb_config,
#     device_map='auto'
# )

# # Configure generation for Qwen
# qwen_gen = qwen_model.generation_config
# qwen_gen.max_new_tokens = 40
# qwen_gen.eos_token_id = qwen_tokenizer.eos_token_id
# qwen_gen.pad_token_id = qwen_tokenizer.eos_token_id
# qwen_gen.temperature = None
# qwen_gen.num_return_sequences = 1

# print(qwen_model)

In [ ]:
llama_card = "meta-llama/Llama-3.1-8B-Instruct"

llama_tokenizer = AutoTokenizer.from_pretrained(llama_card)
llama_tokenizer.padding_side = "left"
llama_tokenizer.padding_token = llama_tokenizer.eos_token

llama_model = AutoModelForCausalLM.from_pretrained(
    llama_card,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto'
)

llama_gen = llama_model.generation_config
llama_gen.max_new_tokens = 40
llama_gen.eos_token_id = llama_tokenizer.eos_token_id
llama_gen.pad_token_id = llama_tokenizer.eos_token_id
llama_gen.temperature = None
llama_gen.num_return_sequences = 1

print(llama_model)

# [Task 2 - 1.0 points] Prompt setup

Prompting requires an input pre-processing phase where we convert each input example into a specific instruction prompt.


### Prompt Template

Use the following prompt template to process input texts.

In [ ]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as not-sexist
         or sexist. If sexist, classify input text according to one
         of the following four categories: threats, derogation,
         animosity, prejudiced discussion.

         Below you find sexist categories definitions:
         Threats: the text expresses intent or desire to harm a woman.
         Derogation: the text describes a woman in a derogative manner.
         Animosity: the text contains slurs or insults towards a woman.
         Prejudiced discussion: the text expresses supports for
         mistreatment of women as individuals.

         Respond only by writing one of the following categories:
         not-sexist, threats, derogation, animosity, prejudiced.

        TEXT: {text}

        ANSWER:
        """
    }
]

### Instructions

In order to get Task 2 points, we require you to:

* Write a ``prepare_prompts`` function as the one reported below.

### Our implementation

In [ ]:
def prepare_prompts(texts, prompt_template):
  """
    This function format input text samples into instructions prompts.

    Inputs:
      texts: input texts as a list of strings to classify via prompting
      prompt_template: the prompt template provided in this assignment

    Outputs:
      input texts to classify in the form of instruction prompts
  """

  formatted_prompts = []
  for text in texts:
    formatted_prompts.append(prompt_template.format(text=text))

  return formatted_prompts

Below we visualize the effects of the `prepare_prompt` function using both the tokenizers. We can see the differences between the two tokenizers in how they organize the prompt adding special tokens and additional information as the cutting knowledge date of the llama model.

In [ ]:
mistral_prompt = mistral_tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    add_generation_prompt=True,
)

llama_prompt = llama_tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    add_generation_prompt=True,
)

sample_texts = df['text'].values[:1]
print(sample_texts)
prepared = prepare_prompts(sample_texts, llama_prompt)
for i in range(len(sample_texts)):
  print(prepared[i])

prepared = prepare_prompts(sample_texts, mistral_prompt)
for i in range(len(sample_texts)):
  print(prepared[i])

### Notes

1. You are free to modify the prompt format (**not its content**) as you like depending on your code implementation.

2. Note that the provided prompt has placeholders. You need to format the string to replace placeholders. Huggingface might have dedicated APIs for this.

# [Task 3 - 1.0 points] Inference

We are now ready to define the inference loop where we prompt the model with each pre-processed sample.

### Instructions

In order to get Task 3 points, we require you to:

* Write a ``generate_responses`` function as the one reported below.
* Write a ``process_response`` function as the one reported below.

### Our implementation

First, we define a function that based on the used model provides a response extractor function that cleans the answer of the LLM outputing a single label category, as required by the classification task we are tackling.

In [ ]:
import re

def extract_response_warped(model_name):

  '''
  The function returns a function based on the model we are using. The returned function processes the raw response of the model extracting only the label category.

  Inputs:
    model_name: string with the name of the model we are using
        response: input of the internal function and is the raw response of the model

  Outputs:
    extract_response: the function that cleans the answer
        The internal function returns the label category of the answer.
  '''

  model_name = model_name.lower()

  if model_name=='mistral':
    def extract_response(response):
      match = [m for m in re.finditer('ANSWER:', response)][-1]
      parsed = response[match.end():].strip()
      label_only = parsed.split('(')[0].strip()

      # The model outputs prejudiced discussion as indicated in the prompt definitions, we change to prejudiced to be coherent with the labels
      if label_only == 'prejudiced discussion':
        label_only = 'prejudiced'

      # The model outputs in some cases more categories. Select the first one in this case
      if ',' in label_only:
        label_only = label_only.split(',')[0]

      return label_only

  elif model_name=='llama':
    def extract_response(response):
      LABEL_PATTERN = re.compile(
        r"(not-sexist|threats|derogation|animosity|prejudiced)",
        flags=re.IGNORECASE
      )
      matches = LABEL_PATTERN.findall(response)
      return matches[-1]

  return extract_response

Below we implement the two recquired functions.

In [ ]:
def generate_responses_batched(model, tokenizer, prompt_examples, model_name):
  """
    This function implements the inference loop for a LLM model.
    Given a set of examples, the model is tasked to generate
    a response.

    Inputs:
      model: LLM model instance for prompting
      tokenizer: the tokenizer corresponding to the model
      prompt_examples: pre-processed text samples in the form of a DataLoader
      model_name: string with the name of the model we are using,
                  necessary to get the cleaning function

    Outputs:
      generated responses
  """

  responses = []

  with torch.inference_mode():
    for batch_text, batch_label in tqdm(prompt_examples, desc="Inference"):

      input_ids = batch_text['input_ids'].to(model.device)
      attention_mask = batch_text['attention_mask'].to(model.device)

      response = model.generate(
          input_ids=input_ids,
          attention_mask=attention_mask,
          generation_config=model.generation_config,
          do_sample=False
      )
      raw_response = tokenizer.batch_decode(response,
                                            skip_special_tokens=True)
      extractor = extract_response_warped(model_name)
      raw_response = [extractor(resp) for resp in raw_response]
      responses.extend(raw_response)
  return responses


In [ ]:
def process_response(response):
  """
    This function takes a textual response generated by the LLM
    and processes it to map the response to a binary label.

    Inputs:
      response: generated response from LLM

    Outputs:
      parsed classification response.
      Use the following mapping:
      {
        'not-sexist': 0,
        'threats': 1,
        'derogation': 2,
        'animosity': 3,
        'prejudiced': 4
      }
  """

  text2class = {
      'not-sexist': 0,
      'threats': 1,
      'derogation': 2,
      'animosity': 3,
      'prejudiced': 4
  }
  return text2class[response]

We define below the collate function that will be passed to the `DataLoader` instance. Moreover we add two columns to the dataset where the prepared prompts for each model are stored and create the `DataLoader` instances.

In [ ]:
def collate_fn(tokenizer):

  def process(batch):
    texts = tokenizer.batch_encode_plus(
        [ex['text'] for ex in batch],
        return_tensors='pt',
        padding=True,
        truncation=True
    )
    labels = torch.tensor([ex['label'] for ex in batch])
    return {'input_ids': texts['input_ids'],
            'attention_mask': texts['attention_mask']}, labels

  return process

In [ ]:
# Create new columns in the dataframe for the framed texts
df['framed_text_mistral'] = prepare_prompts(df['text'].values, mistral_prompt)
df['framed_text_llama'] = prepare_prompts(df['text'].values, llama_prompt)
df['label'] = df.apply(lambda x: process_response(x['label_category']), axis=1)

mistral_data = Dataset.from_pandas(df[['framed_text_mistral', 'label']].rename(
    columns={'framed_text_mistral': 'text'}
))
llama_data = Dataset.from_pandas(df[['framed_text_llama', 'label']].rename(
    columns={'framed_text_llama': 'text'}
))

print(mistral_data)
print(llama_data)

mistral_loader = torch.utils.data.DataLoader(mistral_data,
                                             batch_size=2,
                                             shuffle=False,
                                             collate_fn=collate_fn(mistral_tokenizer))

llama_loader = torch.utils.data.DataLoader(llama_data,
                                          batch_size=2,
                                          shuffle=False,
                                          collate_fn=collate_fn(llama_tokenizer))
df.head()

Now we can use the models to generate responses, storing them in two separate lists and visualizing the distribution of outputed classes from the models. We can already notice that the mistral model heavily prefers the *animosity* class.

In [ ]:
mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
llama_tokenizer.pad_token = llama_tokenizer.eos_token

mistral_answers = generate_responses_batched(mistral_model, mistral_tokenizer, mistral_loader, 'mistral')
llama_answers = generate_responses_batched(llama_model, llama_tokenizer, llama_loader, 'llama')

In [ ]:
print(np.unique(llama_answers, return_counts=True))
print(np.unique(mistral_answers, return_counts=True))

We performed a simple comparison of answers looking computing the percentage of disagreement between the two models, finding that it's pretty high: 51.33%.

In [ ]:
answers = pd.DataFrame({
            "True Answer": df['label_category'],
            "Mistral": mistral_answers,
            "Llama": llama_answers
          })

print(f"Percentage of disagreements between the models: {((answers['Mistral'] != answers['Llama']).sum() / 3):.2f}%")
disagreeing_answers = answers[answers['Mistral'] != answers['Llama']]
disagreeing_answers.head(10)

In order to save computing resources we save in a persistent way, as a json file, the answers, so that they can be used without reloading and running the models.

In [ ]:
def save_answers(answers1, answers2, model_names = ['Mistral', 'Llama'], filename='zero_shot_answers.json'):
    drive.mount('/content/drive')
    path = f'/content/drive/MyDrive/NLP_tutorials/{filename}'

    data = {model_names[0]: answers1, model_names[1]: answers2}

    with open(path, 'w') as f:
        json.dump(data, f, indent=2)

save_answers(mistral_answers, llama_answers)

## Notes

1. According to our tests, it should take you ~10 mins to perform full inference on 300 samples on Colab.

# [Task 4 - 0.5 points] Metrics

In order to evaluate selected LLMs, we need to compute performance metrics.

We compute **macro F1-score** and the ratio of failed responses generated by models (**fail-ratio**).

That is, how frequent the LLM fails to follow instructions and provides incorrect responses that do not address the classification task.

In summary, we parse generated responses as follows:
- **0** if 'not-sexist'
- **1** if 'threats'
- **2** if 'derogation'
- **3** if 'animosity'
- **4** if 'prejudiced'
- **0** if the model does not answer in either way

### Instructions

In order to get Task 4 points, we require you to:

* Write a ``compute_metrics`` function as the one reported below.
* Compute metrics for the two selected LLMs.

### Our implementation

In [ ]:
def safe_process_response(pred):
  '''
  This function is responsible to guarantee that every llm's predictions that is
  not the class label is mapped to 0, as recquired.

  Inputs:
    pred: prediction of the model, the string answer

  Outputs:
    integer corresponding to the predicted category
  '''
  try:
    return process_response(pred)
  except:
    return 0

def compute_metrics(y_pred, y_true):
  """
    This function takes predicted and ground-truth labels and compute
    metrics. In particular, this function compute accuracy and
    fail-ratio metrics. This function internally invokes
    `process_response` to compute metrics.

    Inputs:
      y_pred: parsed LLM responses
      y_true: ground-truth binary labels

    Outputs:
      dictionary containing desired metrics
  """
  f1_score = evaluate.load('f1')
  acc_score = evaluate.load('accuracy')

  y_pred = [safe_process_response(pred) for pred in y_pred]

  y_pred = np.array(y_pred)
  y_true = np.array(y_true)

  f1 = f1_score.compute(predictions=y_pred, references=y_true, average='macro')
  acc = acc_score.compute(predictions=y_pred, references=y_true)
  fail_ratio = 1 - acc['accuracy']

  return {**f1, **{'fail ratio': fail_ratio}}

Below we defined the function that loads the lists of answers produced by the models. Those answers we saved in the Inference section. After it the metrics are computed and visualized. We can see that the Llama model outperfroms by a significant margin the Mistral one.

In [ ]:
def load_answers(filename='zero_shot_answers.json'):
    drive.mount('/content/drive', force_remount=False)
    path = f'/content/drive/MyDrive/NLP_tutorials/{filename}'

    with open(path, 'r') as f:
        data = json.load(f)

    return data["Mistral"], data["Llama"]

In [ ]:
mistral_answers, llama_answers = load_answers()
mistral_metrics = compute_metrics(mistral_answers, df['label'].values)
llama_metrics = compute_metrics(llama_answers, df['label'].values)
results = {
    "Model": ["Mistral", "Llama"],
    "F1-score": [mistral_metrics['f1'], llama_metrics['f1']],
    "Fail Ratio": [mistral_metrics['fail ratio'], llama_metrics['fail ratio']]
}
results_df = pd.DataFrame(results)
print(tabulate(results_df, headers='keys', tablefmt='fancy_grid', showindex=False))

# [Task 5 - 1.0 points] Few-shot Inference

So far, we have tested models in a zero-shot fashion: we provide the input text to classify and instruct the model to generate a response.

We are now interested in performing few-shot prompting to see the impact of providing demonstration examples.

To do so, we slightly change the prompt template as follows.

In [ ]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as not-sexist
         or sexist. If sexist, classify input text according to one
         of the following four categories: threats, derogation,
         animosity, prejudiced discussion.

         Below you find sexist categories definitions:
         Threats: the text expresses intent or desire to harm a woman.
         Derogation: the text describes a woman in a derogative manner.
         Animosity: the text contains slurs or insults towards a woman.
         Prejudiced discussion: the text expresses supports for
         mistreatment of women as individuals.

         Respond only by writing one of the following categories:
         not-sexist, threats, derogation, animosity, prejudiced.

        EXAMPLES: {examples}

        TEXT: {text}

        ANSWER:
        """
    }
]

The new prompt template reports some demonstration examples to instruct the model.

Generally, we provide an equal number of demonstrations per class as shown in the example below. We have commented out this line in order to avoid overriding the correct prompt when running everything together.

In [ ]:
# prompt = [
#     {
#         'role': 'system',
#         'content': 'You are an annotator for sexism detection.'
#     },
#     {
#         'role': 'user',
#         'content': """Your task is to classify input text as not-sexist
#          or sexist. If sexist, classify input text according to one
#          of the following four categories: threats, derogation,
#          animosity, prejudiced discussion.

#          Below you find sexist categories definitions:
#          Threats: the text expresses intent or desire to harm a woman.
#          Derogation: the text describes a woman in a derogative manner.
#          Animosity: the text contains slurs or insults towards a woman.
#          Prejudiced discussion: the text expresses supports for
#          mistreatment of women as individuals.

#          Respond only by writing one of the following categories:
#          not-sexist, threats, derogation, animosity, prejudiced.

#          EXAMPLES:
#          TEXT: **example 1**
#          ANSWER: threats
#          TEXT: **example 2**
#          ANSWER: not-sexist

#          TEXT: {text}

#         ANSWER:
#         """
#     }
# ]

## Instructions

In order to get Task 5 points, we require you to:

- Load ``demonstrations.csv`` and encode it into a ``pandas.DataFrame`` object.
- Define a ``build_few_shot_demonstrations`` function as the one reported below.
- Modify ``prepare_prompts`` to support demonstrations.
- Perform few-shot inference as in Task 3.
- Compute metrics as in Task 4.

### Our implementation

Similarly as before we can see that this dataset is also perfectly balanced.

In [ ]:
demonstrations = pd.read_csv(f"{data_path}/demonstrations.csv")

print(demonstrations['label_category'].value_counts())
print(demonstrations['label_sexist'].value_counts())
demonstrations.head()

For the few-shot inference we have developed two different pipelines. The first one is the simple implementation that statically adds the demonstrations to the prompt while the second includes those examples that are deemed to be the most similar to the text we want to classify.

We can thus, estimate the importance of carefully deciding which examples to include in the prompt.

#### Simple implementation

In this section we implement few-shot inference in a very simple way. The examples to include in the prompt are sampled without considering the current text sample. We randomly choose `num_per_class` demonstration data for every class and add them to the prompt *examples* section.

In [ ]:
def build_few_shot_demonstrations(demonstrations, num_per_class=2):
  """
    Inputs:
      demonstrations: DataFrame wrapping demonstrations.csv
      num_per_class: number of demonstrations per class

    Outputs:
      list of demonstrations to inject into the prompt template.
  """
  sampled = (demonstrations
             .groupby('label_category')
             .sample(n=num_per_class)
             .reset_index(drop=True))

  examples = [(row['text'], row['label_category']) for _, row in sampled.iterrows()]
  return examples

In [ ]:
def prepare_prompts(texts, examples, prompt_template):
  """
    This function format input text samples into instructions prompts.

    Inputs:
      texts: input texts to classify via prompting
      examples: list of tuples - demonstrations to inject into the prompt template
      prompt_template: the prompt template provided in this section of the assignments

    Outputs:
      input texts to classify in the form of instruction prompts
  """

  examples = '\n'.join([f"TEXT:{text}\n ANSWER: {label}"
                        for (text, label) in examples])
  formatted_prompts = []
  for text in texts:
    formatted_prompts.append(prompt_template.format(text=text, examples=examples))

  return formatted_prompts

Below we visualize the generated prompts one adding the demonstrations and the text to classify.

In [ ]:
demonstrations_list = build_few_shot_demonstrations(demonstrations, num_per_class=2)

copied = df.copy()

# Create new columns in the dataframe for the framed texts
copied['few_shot_mistral'] = prepare_prompts(df['text'].values, demonstrations_list, mistral_prompt)
copied['few_shot_llama'] = prepare_prompts(df['text'].values, demonstrations_list, llama_prompt)

print(copied['few_shot_mistral'][0])
print(copied['few_shot_llama'][0])

In the following snippets we define a function that performs the inference and computes metrics for the provided models and for different `num_per_class` demonstrations and then we execute it.

In [ ]:
def compare_few_shots(models, tokenizers, prompt_template, text_data, demonstration_data, num_per_class=2):

  '''
  The function performs inference of the models (for different number of examples per class
  on the provided data outputing metrices and model's answers.

  Inputs:
    models: list of models to run inference on
    tokenizers: list of the tokenizers corresponding to the models
    prompt_template: list of prompt templates provided in this assignment already tokenized with apply_chat_template
    text_data: DataFrame containing the text to classify
    demonstration_data: DataFrame containing the possible demonstrations
    num_per_class: number of demonstrations per class, loop on that

  Output:
    Tuple containing a dictionary with computed metrics and a dictionary with model's answers
  '''

  results = {}
  answers = {}

  if not isinstance(num_per_class, list):
    num_per_class = [num_per_class]

  for num in num_per_class:
    print(f"Building demonstrations with number per class: {num}")
    demonstrations = build_few_shot_demonstrations(demonstration_data, num)
    model_name = 'Mistral'
    for i, (model, tokenizer) in enumerate(zip(models, tokenizers)):
      if i == 1:
        model_name = 'Llama'
      print(f"\t...Generating responses for {model_name}...")
      prompted_examples = prepare_prompts(text_data['text'].values, demonstrations, prompt_template[i])
      prompted_data = Dataset.from_pandas(pd.DataFrame({'text':prompted_examples, 'label': text_data['label'].values}))
      prompted_loader = torch.utils.data.DataLoader(prompted_data,
                                          batch_size=2,
                                          shuffle=False,
                                          collate_fn=collate_fn(tokenizer))
      response = generate_responses_batched(model, tokenizer, prompted_loader, model_name)
      print(f"\t...Computing metrics for {model_name}...")
      metrics = compute_metrics(response, text_data['label'].values)
      results[f'{model_name} {num}'] = metrics
      answers[f'{model_name} {num}'] = response

  return results, answers

In [ ]:
# Create the two prompts with apply_chat_template
mistral_prompt = mistral_tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    add_generation_prompt=True,
)
llama_prompt = llama_tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    add_generation_prompt=True,
)

# Call the function to evaluate all models
results, answers = compare_few_shots([mistral_model, llama_model],
                                     [mistral_tokenizer, llama_tokenizer],
                                     [mistral_prompt, llama_prompt],
                                     df,
                                     demonstrations,
                                     [2,4,6])

Similarly as before also in this case we save the answers and the results in a json file, facilitating the exploration of the results without the need of re-running everything.

In [ ]:
def save_json(results, answers, filename='simple_few_shot_results.json'):

  drive.mount('/content/drive')
  path = f'/content/drive/MyDrive/NLP_tutorials/{filename}'
  with open(path, 'w') as f:
    json.dump({'results': results, 'answers': answers}, f, indent=2)

save_json(results, answers)

#### Similarity based

In the following section we implement few-shot inference considering the cosine similarity between the embeddings of the demonstration examples and the text we are interested in classifying. The most similar demonstrations for each class are then selected and used in the prompt.

For the first thing we load the encoder that will be used to compute embeddings of the demonstration examples. We chose the `BAAI/bge-large-en-v1.5` encoder. We didn't use the internal representations of the two models used to predict classes to improve on efficiency.

In [ ]:
from sentence_transformers import SentenceTransformer

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

encoder_card = 'BAAI/bge-large-en-v1.5'
encoder_model = SentenceTransformer(encoder_card)


The following code snippet computes the embeddings of the every example in the demonstration dataset. The lines are commented out because we have already saved it in a persistent way.

To recompute it uncomment the lines.

In [ ]:
# # Precompute the embeddings and store them
# embeddings = encoder_model.encode(demonstrations['text'].values,
#                                   batch_size=32,
#                                   convert_to_tensor=True,
#                                   normalize_embeddings=True,
#                                   show_progress_bar=True)

# def save_embeddings(embeddings, filename='embeddings.npy'):
#   drive.mount('/content/drive')
#   path = f'/content/drive/MyDrive/NLP_tutorials/{filename}'
#   # Corrected arguments order and conversion to numpy array for np.save
#   np.save(path, embeddings.cpu().numpy())

# save_embeddings(embeddings)

With the following utility function we inspect the most and less similar examples for some text samples. We can in fact notice that some similarity between the texts is present:
- the second example is paired with a text about shooting the girl, which is exactly what the text is about.
- the third example is paired with a text about weight and fatness which is also a topic in the text.

In [ ]:
def report_similarity(
    model: SentenceTransformer,
    corpus_texts: list[str],
    corpus_embeddings: np.ndarray,
    query_texts: list[str],
    top_k: int = 1
):
    """
    Prints the most similar and least similar texts from a corpus
    for each query text.

    Args:
        model: A SentenceTransformer instance.
        corpus_texts: List of texts corresponding to precomputed embeddings.
        corpus_embeddings: np.ndarray of shape (N, D)
        query_texts: List of new texts to compare to the corpus.
        top_k: Number of most-similar results to show (default = 1)
    """

    # Compute embeddings for the new query samples
    query_embeddings = model.encode(query_texts, convert_to_tensor=True)

    # Convert corpus to tensor if not already
    corpus_tensor = torch.tensor(corpus_embeddings, dtype=torch.float32).to(device)

    for query, q_emb in zip(query_texts, query_embeddings):
        print(f"\n==============================")
        print(f"QUERY: {query}")
        print("==============================")

        # Compute cosine similarities
        similarities = (corpus_tensor @ q_emb).cpu().numpy()

        # Most similar
        top_indices = np.argsort(similarities)[::-1][:top_k]
        for rank, idx in enumerate(top_indices, start=1):
            print(f"\nTOP {rank} MOST SIMILAR:")
            print(f"Text: {corpus_texts[idx]}")
            print(f"Score: {similarities[idx]:.4f}")

        # Least similar
        worst_idx = np.argmin(similarities)
        print("\nLEAST SIMILAR:")
        print(f"Text: {corpus_texts[worst_idx]}")
        print(f"Score: {similarities[worst_idx]:.4f}")

In [ ]:
embeddings = np.load(f"/content/drive/MyDrive/NLP_tutorials/embeddings.npy")

In [ ]:
query_texts = df["text"].sample(n=3, random_state=42).tolist()
report_similarity(encoder_model, demonstrations['text'].values, embeddings, query_texts)

The remaining parts of this sub-section mirror what we did in the previous sub-section. We implement:
- function to extract the correct examples to add to the prompt;
- function to prepare prompts;
- function that executes the models for different values of `num_per_class` and computes the metrics.

We finally call the last function and save the results.

In [ ]:
def build_similarity_few_shot_demonstration(text, demonstrations, embeddings, num_per_class=2):

  """
  The function builds the demonstrations to inject into the prompt template looking at the cosine similarity.

    Inputs:
      text: input text, with respect to which we should build the demonstrations
      demonstrations: DataFrame wrapping demonstrations.csv
      embeddings: numpy array with precomputed embeddings of the demonstrations.csv texts
      num_per_class: number of demonstrations per class

    Outputs:
      list of demonstrations to inject into the prompt template.
  """

  # Build the embedding of the text
  text_embedding = encoder_model.encode(text, convert_to_tensor=True, normalize_embeddings=True)

  # Compute cosine similarity with the precomputed embeddings
  # Move emb_tensor to the same device as text_embedding
  emb_tensor = torch.tensor(embeddings, dtype=torch.float32).to(text_embedding.device)
  similarities = (emb_tensor @ text_embedding).cpu().numpy()

  # Choose num_per_class most similar examples for every class
  demonstrations = demonstrations.copy()
  demonstrations["similarity"] = similarities

  # Return the list of selected prompts
  selected = (demonstrations
              .sort_values("similarity", ascending=False)
              .groupby("label_category")
              .head(num_per_class))

  examples = [(row['text'], row['label_category']) for _, row in selected.iterrows()]
  return examples

In [ ]:
def similarity_prepare_prompts(texts, examples, prompt_template, embeddings, num_per_class=2):
  """
    This function format input text samples into instructions prompts.

    Inputs:
      texts: input texts to classify via prompting
      examples: dataframe that warps demonstrations.csv
      prompt_template: the prompt template provided in this assignment
      embeddings: numpy array with precomputed embeddings of the demonstrations.csv texts

    Outputs:
      input texts to classify in the form of instruction prompts
  """
  formatted_prompts = []
  for text in texts:
    examples_for_text = build_similarity_few_shot_demonstration(text,
                                                                 examples,
                                                                 embeddings,
                                                                 num_per_class=num_per_class)
    examples_for_text = '\n'.join([f"TEXT:{text}\n ANSWER: {label}"
                                  for (text, label) in examples_for_text])
    formatted_prompts.append(prompt_template.format(text=text, examples=examples_for_text))

  return formatted_prompts

In [ ]:
def compare_similarity_few_shots(models, tokenizers, prompt_template, text_data, demonstration_data, embeddings, num_per_class=2):

  '''
  The function performs inference of the models (for different number of examples per class
  on the provided data outputing metrices and model's answers.

  Inputs:
    models: list of models to run inference on
    tokenizers: list of the tokenizers corresponding to the models
    prompt_template: list of prompt templates provided in this assignment already tokenized with apply_chat_template
    text_data: DataFrame containing the text to classify
    demonstration_data: DataFrame containing the possible demonstrations
    num_per_class: number of demonstrations per class, loop on that

  Output:
    dictionary with metrics
    dictionary with model's answers
  '''

  results = {}
  answers = {}

  if not isinstance(num_per_class, list):
    num_per_class = [num_per_class]

  for num in num_per_class:
    print(f"Number of samples per class: {num}")
    model_name = 'Mistral'
    for i, (model, tokenizer) in enumerate(zip(models, tokenizers)):
      if i == 1:
        model_name = 'Llama'
      print(f"\t...Generating responses for {model_name}...")
      prompted_examples = similarity_prepare_prompts(text_data['text'].values, demonstrations, prompt_template[i], embeddings, num)
      prompted_data = Dataset.from_pandas(pd.DataFrame({'text':prompted_examples, 'label': text_data['label'].values}))
      prompted_loader = torch.utils.data.DataLoader(prompted_data,
                                          batch_size=2,
                                          shuffle=False,
                                          collate_fn=collate_fn(tokenizer))
      response = generate_responses_batched(model, tokenizer, prompted_loader, model_name)
      print(f"\t...Computing metrics for {model_name}...")
      metrics = compute_metrics(response, text_data['label'].values)
      results[f'{model_name} {num}'] = metrics
      answers[f'{model_name} {num}'] = response

  return results, answers

In [ ]:
# Create the two prompts with apply_chat_template
mistral_prompt = mistral_tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    add_generation_prompt=True,
)
llama_prompt = llama_tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    add_generation_prompt=True,
)

# Call the function to evaluate all models
results, answers = compare_similarity_few_shots([mistral_model, llama_model],
                                     [mistral_tokenizer, llama_tokenizer],
                                     [mistral_prompt, llama_prompt],
                                     df,
                                     demonstrations,
                                     embeddings,
                                     [2,4,6])

save_json(results, answers, 'similarity_few_shot_results.json')

## Notes

1. You are free to pick any value for ``num_per_class``.

2. According to our tests, few-shot prompting increases inference time by some minutes (we experimented with ``num_per_class`` $\in [2, 4]$).

# [Task 6 - 1.0 points] Error Analysis

We are now interested in evaluating model responses and comparing their performance.

This analysis helps us in understanding

- Classification task performance gap: are the models good at this task?
- Generation quality: which kind of responses do models generate?
- Errors: which kind of mistakes do models do?

### Instructions

In order to get Task 6 points, we require you to:

* Compare classification performance of selected LLMs in a Table.
* Compute confusion matrices for selected LLMs.
* Briefly summarize your observations on generated responses.

### Our implementation

In this section we compare:
- The models in zero-shot inference;
- The models in few-shot inference;
- The benefits of few-shot inference;
- The two different strategies to include demonstrations in the prompt;
- The effect of num_per_class on performance.

#### Utility functions

In [ ]:
def load_json(filename='results.json'):

    '''
    This function is responsible for loading the results and answers we previously saved
    '''

    drive.mount('/content/drive', force_remount=False)
    path = f'/content/drive/MyDrive/NLP_tutorials/{filename}'
    with open(path, 'r') as f:
        data = json.load(f)
    results = data.get('results', {})
    answers = data.get('answers', {})

    return results, answers

The three functions defined in the following code snippet are used to visualize the confusion matrices and the precomputed metrics of the models. Moreover, we also additionaly implement a function that computes per class precision and recall.

In [ ]:
def show_confusion_matrix(answers, y_true, labels=None, normalize='true'):

  for model_name, model_answers in answers.items():
    cm = confusion_matrix(y_true, model_answers, labels=labels, normalize=normalize)
    sns.heatmap(cm,
                annot=True,
                fmt='.2f',
                cmap='Blues',
                xticklabels=labels,
                yticklabels=labels)
    plt.title(f'{model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.show()

def show_confusion_matrix_two_sets(answers1, answers2, y_true, labels=None, titles=("Random", "Similarity"), normalize='true'):

  model_names = list(answers1.keys())

  for model_name in model_names:
    model_answers1 = answers1[model_name]
    model_answers2 = answers2[model_name]
    cm1 = confusion_matrix(y_true, model_answers1, labels=labels, normalize=normalize)
    cm2 = confusion_matrix(y_true, model_answers2, labels=labels, normalize=normalize)
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    # Left
    sns.heatmap(
            cm1,
            annot=True,
            cmap='Blues',
            xticklabels=labels,
            yticklabels=labels,
            fmt='.2f',
            ax=axes[0]
        )
    axes[0].set_title(f"{model_name} — {titles[0]}")
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("True")

    # Right
    sns.heatmap(
            cm2,
            annot=True,
            cmap='Blues',
            xticklabels=labels,
            yticklabels=labels,
            fmt='.2f',
            ax=axes[1]
        )
    axes[1].set_title(f"{model_name} — {titles[1]}")
    axes[1].set_xlabel("Predicted")
    axes[1].set_ylabel("True")

    plt.tight_layout()
    plt.show()

def build_results(zeroA, zeroB, random_few_shot, similarity_few_shot):

    results = {
        "Mistral": {
            "zero_shot": zeroA,
            "simple_few_shot": {},
            "similarity_few_shot": {}
        },
        "Llama": {
            "zero_shot": zeroB,
            "simple_few_shot": {},
            "similarity_few_shot": {}
        }
    }

    # Simple few-shot
    for key, metrics in random_few_shot.items():
        model_name, shots_str = key.split()
        shots = int(shots_str)

        results[model_name]["simple_few_shot"][shots] = metrics

    # Similarity few-shot
    for key, metrics in similarity_few_shot.items():
        model_name, shots_str = key.split()
        shots = int(shots_str)

        results[model_name]["similarity_few_shot"][shots] = metrics

    return results

def summarize_results(results):
    """
    Create the comparison table:
    Mode | Shots | Mistral F1 | Mistral Fail | Llama F1 | Llama Fail
    """
    rows = []

    rows.append({
        "Mode": "zero_shot",
        "Shots": 0,
        "Mistral F1": results["Mistral"]["zero_shot"]["f1"],
        "Mistral Fail": results["Mistral"]["zero_shot"]["fail ratio"],
        "Llama F1": results["Llama"]["zero_shot"]["f1"],
        "Llama Fail": results["Llama"]["zero_shot"]["fail ratio"],
    })

    for shots in sorted(results["Mistral"]["simple_few_shot"].keys()):
        rows.append({
            "Mode": "random_few_shot",
            "Shots": shots,
            "Mistral F1": results["Mistral"]["simple_few_shot"][shots]["f1"],
            "Mistral Fail": results["Mistral"]["simple_few_shot"][shots]["fail ratio"],
            "Llama F1": results["Llama"]["simple_few_shot"][shots]["f1"],
            "Llama Fail": results["Llama"]["simple_few_shot"][shots]["fail ratio"],
        })

    for shots in sorted(results["Mistral"]["similarity_few_shot"].keys()):
        rows.append({
            "Mode": "similarity_few_shot",
            "Shots": shots,
            "Mistral F1": results["Mistral"]["similarity_few_shot"][shots]["f1"],
            "Mistral Fail": results["Mistral"]["similarity_few_shot"][shots]["fail ratio"],
            "Llama F1": results["Llama"]["similarity_few_shot"][shots]["f1"],
            "Llama Fail": results["Llama"]["similarity_few_shot"][shots]["fail ratio"],
        })

    df = pd.DataFrame(rows)
    print(tabulate(df, headers="keys", tablefmt="fancy_grid", showindex=False))
    return df

from sklearn.metrics import precision_recall_fscore_support

def compute_per_class_precision_recall(y_true, y_pred, classes):
    """
    Compute per-class precision and recall only.

    y_true: list of true labels (strings or ints)
    y_pred: list of predicted labels (strings or ints)
    classes: list of all class labels (strings)

    Returns:
    dict[class] = {"precision": ..., "recall": ...}
    """
    safe_preds = [p if p in classes else None for p in y_pred]

    precision, recall, _, _ = precision_recall_fscore_support(
        y_true, safe_preds, labels=classes, zero_division=0
    )

    per_class = {cls: {"precision": p, "recall": r} for cls, p, r in zip(classes, precision, recall)}
    return per_class

def build_results_per_class(df, zero_preds, few_shot_random, few_shot_similarity, classes):
    """
    Compute per-class precision/recall for all configurations and store in a results dict.

    Parameters:
    - df: dataframe containing 'labels' (int) and 'label_category' (str)
    - zero_preds: dict of zero-shot predictions, e.g. {"Mistral": [...], "Llama": [...]}
    - few_shot_random: dict like {"Mistral 2": [...], "Llama 6": [...]} (pred strings)
    - few_shot_similarity: dict like above
    - classes: list of class strings
    """

    results = {}

    # ----- zero-shot -----
    for model, preds in zero_preds.items():
        results[model] = {}
        results[model]["zero_shot"] = {
            "per_class": compute_per_class_precision_recall(
                df['label_category'].tolist(), preds, classes
            )
        }

    # ----- few-shot modes -----
    for mode_name, few_shot_dict in [("random_few_shot", few_shot_random),
                                     ("similarity_few_shot", few_shot_similarity)]:
        for key, preds in few_shot_dict.items():
            model_name, shots_str = key.split()
            shots = int(shots_str)

            if model_name not in results:
                results[model_name] = {}

            results[model_name].setdefault(mode_name, {})

            results[model_name][mode_name][shots] = {
                "per_class": compute_per_class_precision_recall(
                    df['label_category'].tolist(), preds, classes
                )
            }

    return results

def compute_delta_table(results, model_name, shots=6):
    """
    Compute delta (6-shot - zero-shot) per class for both random and similarity few-shot.

    Returns a DataFrame and prints nicely.
    """
    zero = results[model_name]["zero_shot"]["per_class"]
    rnd = results[model_name]["random_few_shot"][shots]["per_class"]
    sim = results[model_name]["similarity_few_shot"][shots]["per_class"]

    rows = []
    for cls in zero.keys():
        rows.append({
            "Class": cls,
            "Δ Precision (Random)": rnd[cls]["precision"] - zero[cls]["precision"],
            "Δ Recall (Random)": rnd[cls]["recall"] - zero[cls]["recall"],
            "Δ Precision (Similarity)": sim[cls]["precision"] - zero[cls]["precision"],
            "Δ Recall (Similarity)": sim[cls]["recall"] - zero[cls]["recall"],
        })

    df = pd.DataFrame(rows)
    print(f"\n### {model_name} — Per-Class Improvements (Zero-Shot → {shots}-Shot)\n")
    print(tabulate(df, headers="keys", tablefmt="fancy_grid", showindex=False))
    return df

We now load the answers and results and prepare them for the final comparisons and visualizations.

In [ ]:
results_similarity_few_shot, answers_similarity_few_shot = load_json('similarity_few_shot_results.json')
results_simple_few_shot, answers_simple_few_shot = load_json('simple_few_shot_results.json')
mistral_answers, llama_answers = load_answers()

df['label'] = df.apply(lambda x: process_response(x['label_category']), axis=1)
mistral_metrics = compute_metrics(mistral_answers, df['label'].values)
llama_metrics = compute_metrics(llama_answers, df['label'].values)

#### Numerical metrics

In this sub-section we present the metrics in a table form. We can conclude that:
- Llama is better than Mistral in zero-shot inference;
- The performance difference smoothes out when using few-shot prompting. Mistral can be seen to outperform Llama on some experimentation settings;
- Using the similarity based approach in adding demonstrations significantly increases performance;
- While we can see that adding demonstrations helps there are no evident patterns that suggest the best number of examples to add per class.

In [ ]:
results = build_results(mistral_metrics,
                        llama_metrics,
                        results_simple_few_shot,
                        results_similarity_few_shot)
results_df = summarize_results(results)

Below we also look at precision and recall per class to understand how different models behave with different classes. And what classes benefit the most from including more examples.


***Analize the output!!!***

In [ ]:
classes = df['label_category'].unique().tolist()
zero_preds = {"Mistral": mistral_answers, "Llama": llama_answers}
results_per_class = build_results_per_class(df, zero_preds, answers_simple_few_shot, answers_similarity_few_shot, classes)
# Compute delta tables
df_mistral_delta = compute_delta_table(results_per_class, "Mistral", shots=6)
df_llama_delta = compute_delta_table(results_per_class, "Llama", shots=6)

#### Confusion matrices

In the sub-section all the confusion matrices are displayed.

In [ ]:
show_confusion_matrix({'Mistral': mistral_answers, 'Llama': llama_answers}, df['label_category'].values, labels=df['label_category'].unique(), normalize='pred')

In [ ]:
show_confusion_matrix_two_sets(answers_simple_few_shot, answers_similarity_few_shot, df['label_category'].values, labels=df['label_category'].unique(), normalize='pred')

In [ ]:
show_confusion_matrix_two_sets(answers_simple_few_shot, answers_similarity_few_shot, df['label_category'].values, labels=df['label_category'].unique())

# [Task 7 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...

# FAQ

Please check this frequently asked questions before contacting us.

### Model cards

You can pick any open-source model card you like.

We recommend starting from those reported in this assignment.

### Implementation

Everything can be done via ``transformers`` APIs.

However, you are free to test frameworks, such as [LangChain](https://www.langchain.com/), [LlamaIndex](https://www.llamaindex.ai/) [LitParrot](https://github.com/awesome-software/lit-parrot), provided that you correctly address task instructions.

### Task Performance

The task is challenging and zero-shot prompting may show relatively low performance depending on the chosen model.

### Prompt Template

Do not change the provided prompt template.

You are only allowed to change it in case of a possible extension.

### Optimizations

Any kind of code optimization (e.g., speedup model inference or reduce computational cost) is more than welcome!

### Bonus Points

0.5 bonus points are arbitrarily assigned based on significant contributions such as:

- Outstanding error analysis
- Masterclass code organization
- Evaluate A1 dataset and perform comparison
- Perform prompt tuning

Note that bonus points are only assigned if all task points are attributed (i.e., 6/6).

# The End